In [1]:
# Importo dipendenze.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge
from sklearn.kernel_ridge import KernelRidge
from sklearn.model_selection import GridSearchCV

In [39]:
# Metodo che importa i dati da un foglio di formato excel. Per come è strutturato questo file, il modo corretto per caricare i dati è:
def get_data(filename, sheetname):
    dataset = pd.read_excel(
        filename,
        sheet_name=sheetname,
        index_col=3,
        usecols=5,
        skiprows=8,
    ).dropna(axis=1)
    return dataset

# Importo i dati nella variabile gold_daily e prendo solo la colonna relativa agli "Euro", importandola come serie.
gold = get_data("gold_price.xls", "Weekly_EndofPeriod")
gold = gold["Euro"]

In [3]:
# Metodo che mostra il grafico del nostro dataset.
def show_plot(dataset):
    dataset.plot(title="Grafico dell'andamento del prezzo dell'oro dal 1979 al giorno d'oggi", figsize=(12, 4))
    plt.ylabel("Euro")
    plt.show()

# Metodo che mostra il grafico del nostro dataset in un lasso di tempo specifico. 
def show_period_plot(i_date, f_date, dataset):
    dataset[i_date: f_date].plot(style="o-", figsize=(12, 4))
    plt.ylabel("Euro")
    plt.show()

In [4]:
# Creo un nuovo dataset composto dalla differenza del prezzo da una settimana all'altra.
gold_diff = gold.diff()

In [42]:
# Metodo che crea un nuovo dataset con la strategia per predire i valori futuri.
def extract_features(dataset, weeks):
    y = dataset
    X = pd.DataFrame({"col{}".format(i): dataset.shift(i)
        for i in range(1, weeks+1)}).dropna()
    y = dataset.reindex_like(X)
    return X, y

In [43]:
# Estraggo dal nostro dataset, tramite il metodo extract_features un training set e un validation set.
Xt, yt = extract_features(gold_daily_diff["1979":"2015"], 3)
Xv, yv = extract_features(gold_daily_diff["2016":"2018"], 3)

In [7]:
# Metodo che valuta l'accuratezza del modello. Considera la capacità di prevedere correttamente se il prezzo aumenta o diminuisce.
def accuracy(model, X, y):
    return (np.sign(model.predict(X)) == np.sign(y)).mean()

In [44]:
# Addestro un modello di regressione lineare sul training set.
lrm = LinearRegression()
lrm.fit(Xt, yt)
# Testo l'accuratezza sul modello.
lrm_accuracy = accuracy(lrm, Xv, yv)
print("Accuratezza lrm: {:.2%}".format(lrm_accuracy))

Accuratezza lrm: 48.15%


In [60]:
# Addestro un modello di regressione polinomiale di grado 8 sul training set.
prm = Pipeline([
    ("poly", PolynomialFeatures(degree=3, include_bias=False)),
    ("linreg", LinearRegression())
])
prm.fit(Xt, yt)
# Testo l'accuratezza sul modello.
prm_accuracy = accuracy(prm, Xv, yv)
print("Accuratezza prm di grado 8: {:.2%}".format(prm_accuracy))

Accuratezza prm di grado 8: 49.85%


In [53]:
# Addestro un modello di regressione lineare con regolarizzazione sul training set.
rrm = Ridge(alpha=0.1)
rrm.fit(Xt, yt)
# Testo l'accuratezza sul modello.
rrm_accuracy = accuracy(rrm, Xv, yv)
print("Accuratezza rrm : {:.2%}".format(rrm_accuracy))

Accuratezza rrm : 48.15%


In [54]:
# Addestro un modello di regressione polinomiale di grado 4 con regolarizzazione sul training set.
prrm = Pipeline([
    ("poly", PolynomialFeatures(degree=4, include_bias=False)),
    ("ridge", Ridge(alpha=0.1))
])
prrm.fit(Xt, yt)
# Testo l'accuratezza sul modello.
prrm_accuracy = accuracy(prrm, Xv, yv)
print("Accuratezza prrm di grado 4: {:.2%}".format(prrm_accuracy))

Accuratezza prrm di grado 4: 52.16%


In [56]:
# Addestro un modello di regressione con metodo kernel sul training set.
krrm = KernelRidge(alpha=0.1)
krrm.fit(Xt, yt)
# Testo l'accuratezza sul modello.
krrm_accuracy = accuracy(krrm, Xv, yv)
print("Accuratezza krrm: {:.2%}".format(krrm_accuracy))

Accuratezza krrm: 49.07%


In [35]:
# Addestro un modello di regressione con metodo kernel polinomiale di grado 8 sul training set.
krrmp = KernelRidge(alpha=0.1, kernel="poly", degree=8)
krrmp.fit(Xt, yt)
# Testo l'accuratezza sul modello.
krrmp_accuracy = accuracy(krrmp, Xv, yv)
print("Accuratezza krrmp: {:.2%}".format(krrmp_accuracy))

C:\Users\Luca\Anaconda3\lib\site-packages\sklearn\linear_model\ridge.py:154: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "


Accuratezza krrmp: 48.53%


In [36]:
# Addestro un modello di regressione con metodo kernel RBF sul training set.
krrmr = KernelRidge(alpha=0.1, kernel="rbf", gamma=0.01)
krrmr.fit(Xt, yt)
# Testo l'accuratezza sul modello.
krrmr_accuracy = accuracy(krrmr, Xv, yv)
print("Accuratezza krrmr: {:.2%}".format(krrmr_accuracy))

Accuratezza krrmr: 49.37%


In [22]:
# Testo la regressione con metedo kernel di tipo RBF.
gs = GridSearchCV(KernelRidge(kernel="rbf"), {"alpha": [0.01, 0.1], "gamma": [0.01, 0.1, 1]}, accuracy)
gs.fit(Xt, yt)
print("La combinazione migliore dei parametri che mi da un'accuratezza maggiore è: ")
print(gs.best_params_)
print("La miglior accuratezza dopo il tuning dei parametri è: ", gs.best_score_)

La combinazione migliore dei parametri che mi da un'accuratezza maggiore è: 
{'alpha': 0.1, 'gamma': 0.01}
La miglior accuratezza dopo il tuning dei parametri è:  0.5040349252546633


In [59]:
# Testo la regressione con metedo kernel di tipo polinomiale.
gs = GridSearchCV(KernelRidge(kernel="poly"), {"degree": range(1,10)}, accuracy)
gs.fit(Xt, yt)
print("La combinazione migliore dei parametri che mi da un'accuratezza maggiore è: ")
print(gs.best_params_)
print("La miglior accuratezza dopo il tuning dei parametri è: ", gs.best_score_)